In [1]:
import os
import librosa
import math
import json
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Dataset: https://www.kaggle.com/datasets/andradaolteanu/gtzan-dataset-music-genre-classification

# google colab
#DATASET_PATH="/content/drive/MyDrive/Cursos/Valerio_Velardo/Deep_Learning_for_audio/Data/genres_original"
#JS_PATH="/content/drive/MyDrive/Cursos/Valerio_Velardo/Deep_Learning_for_audio/data.json"

# local
DATASET_PATH="G:\My Drive\Cursos\Valerio_Velardo\Deep_Learning_for_audio\Data\genres_original"
JS_PATH="G:\My Drive\Cursos\Valerio_Velardo\Deep_Learning_for_audio\data.json"

os.chdir(DATASET_PATH)

SAMPLE_RATE=22050
DURATION=30 # measured in seconds
SAMPLES_PER_TRACK=SAMPLE_RATE*DURATION

In [4]:
#Store data in JSON
def save_mfcc(dataset_path, json_path, n_mfcc=13, n_fft=2048, hop_lenght=512, num_segments=5):
  # dictionary to store data
  data = {
      "mapping": [],  # way to mapping different genrer on numbers (ej. 0 - 1)
      "mfcc": [],     # input data
      "labels": []    # outputs (target)
  }

  num_samples_per_segment = int(SAMPLES_PER_TRACK/num_segments)

  # debemos asegurar que tenemos número de vectores MfCC por segmento (input equal shape)
  expected_num_mfcc_vectors_per_segment = math.ceil(num_samples_per_segment/hop_lenght) # 1.2 -> 2

  # loop a traves de todos  los generos
  for i, (dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):
    """
    dirpath: ruta de la carpeta actual
    dirnames: lista de los nombres de las subcarpetas dentro de la carpeta
    filnames: lista de los nombres de los archivos dentro de la carpeta
    """
    # print(i)
    # print("dirtpath:{}".format(dirpath))
    # print("dirnames:{}".format(dirnames))
    # print("filenames:{}".format(filenames))
    # print()

    # asegurar que no estamos a nivel raiz (aseguramos que estamos en genre folder)
    if dirpath is not dataset_path:
      # save the semantic label
      dirpath_components = dirpath.split("/") #genre/blues => ["genre","blues"]
      semantic_label = dirpath_components[-1]
      data["mapping"].append(semantic_label)
      print("\nProcessing {}".format(semantic_label))

      # procces files for a specific genre
      for f in filenames:
        # load audio file
        file_path = os.path.join(dirpath, f)
        signal, sr = librosa.load(file_path, sr=SAMPLE_RATE)

        # process segments extracting mfcc and storing data
        for s in range(num_segments):
          start_sample = num_samples_per_segment * s # s=0 -> 0
          finish_sample = start_sample + num_samples_per_segment # s=0 -> num_samples_per_segment

          #Queremos analizar una porcion del audio
          mfcc = librosa.feature.mfcc(y=signal[start_sample:finish_sample],
                                      sr=sr,
                                      n_fft=n_fft,
                                      n_mfcc=n_mfcc,
                                      hop_length=hop_lenght)
          mfcc = mfcc.T

          # store mfcc for segment if it has expected length
          if len(mfcc) == expected_num_mfcc_vectors_per_segment:
            data["mfcc"].append(mfcc.tolist())  # se pasa a list por json format
            data["labels"].append(i-1)
            print("{}, segment:{}".format(file_path, s+1))

  with open(JS_PATH, "w") as fp:
    json.dump(data, fp, indent=4)

In [6]:
if __name__ == "__main__":
  save_mfcc(DATASET_PATH, JS_PATH, num_segments=10)

Se truncaron las últimas líneas 5000 del resultado de transmisión.
/content/drive/MyDrive/Cursos/Valerio_Velardo/Deep_Learning_for_audio/Data/genres_original/disco/disco.00004.wav, segment:7
/content/drive/MyDrive/Cursos/Valerio_Velardo/Deep_Learning_for_audio/Data/genres_original/disco/disco.00004.wav, segment:8
/content/drive/MyDrive/Cursos/Valerio_Velardo/Deep_Learning_for_audio/Data/genres_original/disco/disco.00004.wav, segment:9
/content/drive/MyDrive/Cursos/Valerio_Velardo/Deep_Learning_for_audio/Data/genres_original/disco/disco.00004.wav, segment:10
/content/drive/MyDrive/Cursos/Valerio_Velardo/Deep_Learning_for_audio/Data/genres_original/disco/disco.00000.wav, segment:1
/content/drive/MyDrive/Cursos/Valerio_Velardo/Deep_Learning_for_audio/Data/genres_original/disco/disco.00000.wav, segment:2
/content/drive/MyDrive/Cursos/Valerio_Velardo/Deep_Learning_for_audio/Data/genres_original/disco/disco.00000.wav, segment:3
/content/drive/MyDrive/Cursos/Valerio_Velardo/Deep_Learning_for_